In [1]:
from __future__ import print_function

import numpy as np
import json
from gensim import corpora
import re

from nltk.sem.logic import *

In [2]:
data_path = '/Users/guru/MyResearch/sg/snli/json/snli_inp_1023.json'#'keras-seq2seq/distinct/ku_corpus_char.distinct.txt'
lines = open(data_path)
jdict = json.load(lines)
predicates = []
pre_pre = []
variables = []
pre_var = []
formulas = []
check_dup = set([])

for line in jdict:
    input_text = (jdict[line])['formula']
    formulas.append(input_text)

count_dict = {"and":0,"or":0,"not":0,"exi":0,"all":0}

In [3]:
from nltk.sem.logic import LogicParser
from nltk.sem.logic import *

logic_parser = LogicParser(type_check=False)
def lexpr(formula_str):
    return logic_parser.parse(formula_str)

def normalize_interpretation(expression):
    norm_interp_str = coq_string_expr(expression)
    return norm_interp_str

def coq_string_expr(expression):    
    if isinstance(expression, str):
        expression = lexpr(expression)
    expr_coq_str = ''
    if isinstance(expression, ApplicationExpression):
        expr_coq_str = coq_string_application_expr(expression)
    elif isinstance(expression, AbstractVariableExpression):
        expr_coq_str = coq_string_abstract_variable_expr(expression)
    elif isinstance(expression, LambdaExpression):
        expr_coq_str = coq_string_lambda_expr(expression)
    elif isinstance(expression, QuantifiedExpression):
        quantifier = {'exists' : '*','exist' : '*','all' : '@','forall' : '@'}
        quant = expression.getQuantifier()
        if quant in quantifier:
            quant = nltk2coq_quantifier[expression.getQuantifier()]
        if(quant == '*'):
            count_dict["exi"] += 1
        elif(quant == '@'):
            count_dict["all"] += 1
        expr_coq_str = coq_string_quantified_expr(expression)
    elif isinstance(expression, AndExpression):
        count_dict["and"] += 1
        expr_coq_str = coq_string_and_expr(expression)
    elif isinstance(expression, OrExpression):
        count_dict["or"] += 1
        expr_coq_str = coq_string_or_expr(expression)
    elif isinstance(expression, NegatedExpression):
        count_dict["not"] += 1
        expr_coq_str = coq_string_not_expr(expression)
    elif isinstance(expression, BinaryExpression):
        expr_coq_str = coq_string_binary_expr(expression)
    elif isinstance(expression, Variable):
        expr_coq_str = '%s' % expression
    else:
        expr_coq_str = str(expression)
    return expr_coq_str


coqstr = coq_string_expr

def coq_string_application_expr(expression):
    # uncurry the arguments and find the base function
    if expression.is_atom():
        #is_atom : 原子論理式かどうか
        function, args = expression.uncurry()
        arg_str = ' '.join("%s" % coqstr(arg) for arg in args)
    else:
        #Leave arguments curried
        function = expression.function
        arg_str = "%s" % coqstr(expression.argument)

    function_str = "%s" % coqstr(function)
    parenthesize_function = False
    if isinstance(function, LambdaExpression):
        if isinstance(function.term, ApplicationExpression):
            if not isinstance(function.term.function,
                              AbstractVariableExpression):
                parenthesize_function = True
        elif not isinstance(function.term, BooleanExpression):
            parenthesize_function = True
    elif isinstance(function, ApplicationExpression):
        parenthesize_function = True

    if parenthesize_function:
        function_str = Tokens.OPEN + function_str + Tokens.CLOSE
    
    return Tokens.OPEN + function_str + ' ' + arg_str + Tokens.CLOSE

reserved_s = \
  {'AND' : '&', 'OR' : '|', 'neg' : '-', 'EMPTY' : '', 'TrueP' : 'True'}
  #{'AND' : 'and', 'OR' : 'or', 'neg' : 'not', 'EMPTY' : '', 'TrueP' : 'True'}
def coq_string_abstract_variable_expr(expression):
    expr_str = str(expression.variable)
    if expr_str in reserved_s:
        expr_str = reserved_s[expr_str]
    if not isinstance(expression, FunctionVariableExpression):
        if expr_str == '':
            expr_str = "%s" % expr_str
        else:
            expr_str = "%s" % expr_str
    else:
        expr_str = "%s" % expr_str
        
    if not re.sub(r'_', "",expr_str) in check_dup:
        if expr_str.startswith('_'):
            expr_str = re.sub(r'_', "", expr_str)
            pre_pre.append(expr_str)
            check_dup.add(expr_str)
            #print("pre",expr_str)
        else :
            pre_var.append(expr_str)
            check_dup.add(expr_str)
            #print("var",expr_str)
    return expr_str


def coq_string_lambda_expr(expression):
    variables = [expression.variable]
    term = expression.term
    while term.__class__ == expression.__class__:
        variables.append(term.variable)
        term = term.term
    return Tokens.OPEN + 'fun ' + ' '.join("%s" % coqstr(v) for v in variables) + \
           ' => ' + "%s" % coqstr(term) + Tokens.CLOSE

nltk2coq_quantifier = {'exists' : '*','exist' : '*','all' : '@','forall' : '@'}
                      #{'exists' : 'exists','exist' : 'exists','all' : 'forall','forall' : 'forall'}
def coq_string_quantified_expr(expression):
    variables = [expression.variable]
    term = expression.term
    while term.__class__ == expression.__class__:
        variables.append(term.variable)
        term = term.term
    nltk_quantifier = expression.getQuantifier()
    # Rename quantifiers, according to coq notation. Such renaming dictionary
    # is defined above as "nltk2coq_quantifier". If a rename convention is not
    # available, use the same as in NLTK.
    if nltk_quantifier in nltk2coq_quantifier:
        coq_quantifier = nltk2coq_quantifier[expression.getQuantifier()]
    else:
        coq_quantifier = nltk_quantifier
    return Tokens.OPEN + coq_quantifier + ' ' \
           + ' '.join("%s" % coqstr(v) for v in variables) + \
           ', ' + "%s" % coqstr(term) + Tokens.CLOSE

def coq_string_and_expr(expression):
    first = coqstr(expression.first)
    second = coqstr(expression.second)
    return Tokens.OPEN + 'and ' + first + ' ' + second + Tokens.CLOSE

def coq_string_or_expr(expression):
    first = coqstr(expression.first)
    second = coqstr(expression.second)
    return Tokens.OPEN + 'or ' + first + ' ' + second + Tokens.CLOSE

def coq_string_not_expr(expression):
    term_str = coqstr(expression.term)
    return Tokens.OPEN + 'not ' + term_str + Tokens.CLOSE

def coq_string_binary_expr(expression):
    first = coqstr(expression.first)
    second = coqstr(expression.second)
    return Tokens.OPEN + first + ' ' + expression.getOp() \
            + ' ' + second + Tokens.CLOSE

In [7]:
print("fomulas",len(formulas))
#errorF = open ('error_fomula.txt','w')
for i,formula in enumerate(formulas):
    if i==1:
        raise
    check_dup = set([])
    pre_pre = []
    pre_var = []
    try:
        coq_string_expr(formula)
        predicates.append(pre_pre)
        variables.append(pre_var)
        test_val = pre_var #置換用のテスト
        pre_pre = []
        pre_var = []
    except:
        #import traceback
        #traceback.print_exc()
        #errorF.write(formula+'\n')
        #print(i," : ",formula)
        #print('\n')
        pre_pre = []
        pre_var = []
        continue
#errorF.close()

fomulas 196


RuntimeError: No active exception to reraise

In [54]:
# 単語の辞書作成

pre_dict = corpora.Dictionary(predicates)
#pre_dict.filter_extremes(no_below=4, no_above=0.4)
pre_dict.save_as_text('itemdic_predicates_nofilter_1206.txt')

val_dict = corpora.Dictionary(variables)
#val_dict.filter_extremes(no_below=4, no_above=0.4)
val_dict.save_as_text('itemdic_variables_nofilter_1206.txt')

#fd = open('counter.json', 'w')
#json.dump(count_dict, fd)
#fd.close()



In [30]:
#置換プログラム
test_val = list(set(['x', 'z363', 'True', 'e', 'Subj', 'z364', 'Acc']))
e_sub = {}
z_sub = {}

ice = 0 #index counter
icz = 0 #index counter

for i in test_val:
    matchObj = re.search(r'e0+', i)
    if(matchObj):
        tmp = "e0"+str(ice)
        e_sub[tmp] = i
        ice+=1
        continue
    matchObj = re.search(r'z[0-9]*', i)
    #print(matchObje )
    if matchObj:
        tmp = "z0"+str(icz)
        z_sub[tmp] = i
        icz+=1
    
e_sub = {v:k for k, v in e_sub.items()}
z_sub = {v:k for k, v in z_sub.items()}

#上の辞書を使って置換

#print(test_val)

def substituteString():
    text = "exists x.(_man(x) & exists z363.(_black(z363) & _shirt(z363) & TrueP & exists e.(_in(e,z363) & (Subj(e) = x) & TrueP)) & exists z364.(_golf(z364) & TrueP & exists e.(_play(e) & (Subj(e) = x) & (Acc(e) = z364) & _outside(e) & TrueP)))"
    for k, v in z_sub.items():   # for/if文では文末のコロン「:」を忘れないように
        text = text.replace(k,v)
    print (text)
    
substituteString()

exists x.(_man(x) & exists z00.(_black(z00) & _shirt(z00) & TrueP & exists e.(_in(e,z00) & (Subj(e) = x) & TrueP)) & exists z01.(_golf(z01) & TrueP & exists e.(_play(e) & (Subj(e) = x) & (Acc(e) = z01) & _outside(e) & TrueP)))


In [ ]:
exists x.(_man(x) & exists z363.(_black(z363) & _shirt(z363) & TrueP & exists e.(_in(e,z363) & (Subj(e) = x) & TrueP)) & exists z364.(_golf(z364) & TrueP & exists e.(_play(e) & (Subj(e) = x) & (Acc(e) = z364) & _outside(e) & TrueP)))